<a href="https://colab.research.google.com/github/andersonfedel/DATASUS-SIA-Extract/blob/main/DATASUS_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#Montagem do Drive Google 
from google.colab import drive 
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [13]:
from ftplib import FTP
import numpy as np
import pandas as pd
import os
from pysus.online_data.SIA import download, show_datatypes
from pysus.utilities.readdbc import read_dbc, read_dbc_dbf, dbc2dbf, dbf_to_csvgz
import sys, errno
import time

CACHEPATH = os.getcwd() + "/" #variável com local dos arquivos temporários

start = time.time()

!apt install libffi-dev
!pip install PySUS #Essa instalação demorará cerca de 20 minutos. Porém deixará o ambiente Colab apto a trabalhar com toda a biblioteca PySUS. 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libffi-dev is already the newest version (3.2.1-8).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [14]:
print(f"Preparação do ambiente executada em {round((time.time() - start )/60)} minutos" )

Preparação do ambiente executada em 0 minutos


### Nessa fase baixaremos os arquivos direto do FTP DATASUS

In [15]:
# Configurando endereço FTP dos arquivos SIA DATASUS. 
# Os arquivos baixados ficarão armazenados de forma temporária em CATHEPATH .
# No SIA os arquivos seguem o formato PAUFAAMM.DBC ou PAUFAAMMa.DBC no caso de São Paulo 

ftp = FTP('ftp.datasus.gov.br')
ftp.login()
ftp.cwd('/dissemin/publicos/SIASUS/200801_/Dados')

dir_list = []#Aqui ficarão guardados todos os arquivos disponíveis para download
ftp.dir(dir_list.append)

### A partir daqui os arquivos .DBC baixados ficarão armazenados em uma pasta temporária. Serão apagados assim que finalizada a sessão do Google Colab.

In [16]:
#Pode-se trabalhar as variáveis 'local', 'ano' e 'mes' para baixar os arquivos desejados.
#Exemplo de iteração para baixar vários arquivos:

locais = ["RO", "RJ"]
anos = ["21"]
meses = ["01"]

arquivos = []

for local in locais:
  for ano in anos:
    for mes in meses:
      #filename = f"PA{local}{ano}{mes}.dbc"
      tempfilename = f"PA{local}{ano}{mes}"
      for i in dir_list:
        if tempfilename in i:
          #print(i.split()[-1])
          arquivos.append(i.split()[-1])
      for arquivo in arquivos:
        A = arquivo
        try:
          ftp.retrbinary("RETR " + arquivo ,open(CACHEPATH + A, 'wb').write)
        except IOError as e:
          if e.errno == errno.EPIPE:
            pass

### A próxima célula converterá todos os arquivos .DBC baixados até agora para o formato .parquet, em pastas divididas por estado no Google Drive montado, no formato .../Projeto_DATASUS/UF. 
###Por exemplo: o arquivo do mês de janeiro de 2021 do Acre será armazendo como: .../Projeto_DATASUS/AC/PAAC2101.parquet

In [18]:
#É preciso Google Drive já montado para rodar essas etapas

#Verificação e criação da pasta do projeto 
if not os.path.isdir("/content/gdrive/MyDrive/Projeto_DATASUS"):
  os.mkdir("/content/gdrive/MyDrive/Projeto_DATASUS")

dirPATH = "/content/gdrive/MyDrive/Projeto_DATASUS"

filelist = [] #Lista temporária com registro dos arquivos processados

for file in os.listdir(CACHEPATH): #Iterando sobre arquivos baixados anteriormente
  if str(file).endswith(".dbc"):   
    if len(file) == 12: #Capturando o nome do arquivo no formato PAUFAAMM
      filename = file[:8] 
    else:
      filename = file[:9] 
    if os.path.isdir(dirPATH + "/" + file[2:4]): #Verificação e criação das pastas divididas por estado
      pass
    else: 
      os.mkdir(dirPATH + "/" + file[2:4])

    if filename + ".parquet" not in os.listdir(dirPATH + "/" + file[2:4]): #Checando se o arquivo .parquet ainda não foi criado
      dir = dirPATH + "/" + file[2:4]
      print(f"Criando arquivo {filename}"+".parquet")
      
      #Conversão de .DBC para pandas DataFrame
      dbc2dbf(CACHEPATH + filename + ".dbc",
              os.path.join(str(CACHEPATH + filename) + str(".dbf"))) 
      dbf_to_csvgz(os.path.join(str(CACHEPATH + filename)+ str(".dbf")))
      fn = os.path.join(CACHEPATH + filename + str(".csv.gz"))
      df = pd.read_csv(fn, parse_dates = ['PA_CMP'] )

      #Tratando coluna data de atendimento (conversão para dtype datetime)
      df["PA_CMP"] = df["PA_CMP"].apply(lambda x: x[:4]+"-"+x[4:6]+"-"+"01")
      df["PA_CMP"] = df["PA_CMP"].astype("datetime64[ns]")
      
      #Salvando o arquivo no formato .parquet em diretórios divididos por estado 
      df.astype({'PA_CIDSEC': str, 'PA_CIDCAS': str, 'PA_ETNIA': str,'PA_AUTORIZ':str, 'PA_CBOCOD':str }).to_parquet(dir + "/" + filename + ".parquet") 
      filelist.append(filename)

      #Deletando arquivos temporários para liberação de espaço em disco
      os.remove(str(CACHEPATH) +"/"+ str(filename) + str(".csv.gz"))
      os.remove(str(CACHEPATH) +"/"+ str(filename) + str(".dbf"))
    else:
      print(f"O arquivo {filename}.parquet parece já ter sido criado. Confira nas pastas")

print("Concluído!")
print(f"Arquivos processados: {filelist}")

O arquivo PAES2102.parquet parece já ter sido criado. Confira nas pastas
O arquivo PARJ2102.parquet parece já ter sido criado. Confira nas pastas
O arquivo PARO2101.parquet parece já ter sido criado. Confira nas pastas
O arquivo PAES2101.parquet parece já ter sido criado. Confira nas pastas
O arquivo PARJ2101.parquet parece já ter sido criado. Confira nas pastas
Concluído!
Arquivos processados: []


In [19]:
#Abrindo um arquivo. parquet pelo Pandas e veriricando integridade dos dados
dfteste = pd.read_parquet(dir + "/" + filename + ".parquet")
dfteste

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,...,PA_CODOCO,PA_FLQT,PA_FLER,PA_ETNIA,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C,PA_INE,PA_NAT_JUR
0,2274175,330550,PG,330550,0,0,0,73,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1244
1,2274175,330550,PG,330550,0,0,0,73,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1244
2,2274175,330550,PG,330550,0,0,0,73,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1244
3,2274175,330550,PG,330550,0,0,0,73,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1244
4,2274175,330550,PG,330550,0,0,0,73,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2850703,7050895,330130,PG,330130,0,0,0,2,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1244
2850704,2271672,330120,PG,330120,0,0,0,36,0,M,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,1244
2850705,2272601,330120,PG,330120,0,0,0,5,0,I,...,1,K,0,nan,0.0,0.0,0.0,NaN,NaN,3999
2850706,3024024,330130,PG,330130,0,0,0,39,0,I,...,1,K,0,nan,0.0,0.0,0.0,145003.0,NaN,2062


In [20]:
end = time.time()
print(f"Minutos decorridos: {(end-start)/60}")

Minutos decorridos: 1.6446895043055216
